In [1]:
import pandas as pd
import os
import glob
import datetime

In [2]:
#Library for social network analysis
import networkx as nx
import networkx.algorithms.community as nx_comm
from networkx.algorithms import bipartite
#import bamboolib
import pelote
from ipysigma import Sigma
import numpy as np
#import plotly.express as px


# Chargement des fichiers de données

In [3]:
df = pd.read_csv("../Data/outputs/2026-01-23_list_of_publi_from_bso.csv", sep =",") # contient les publications
dfs = pd.read_csv("../Data/raw/structure/2025/hal_structure.csv", sep =",", dtype={"docid":str}) # Contient les structures
dfa = pd.read_csv("../Data/raw/auteur/2025/hal_auteur.csv", sep =",", dtype={"idHal_i":str}) # Contient les auteurs
df.columns

Index(['id', 'hal_id', 'pub_year', 'authors', 'authhalid_s', 'authhalid_i',
       'orcid', 'struchal_id', 'hal_classif', 'bso_classif', 'pub_type',
       'is_oa'],
      dtype='object')

In [4]:
dfs

,docid,name_s,type_s,acronym_s,parentAcronym_s,parentDocid_i,parentName_s,parentType_s
0,1002699,Centre d’histoire des sociétés Médiévales et M...,laboratory,MéMo,UP8|UPN,11141|116205,Université Paris 8 Vincennes-Saint-Denis|Unive...,institution|institution
1,11141,Université Paris 8 Vincennes-Saint-Denis,institution,UP8,NaN,NaN,NaN,NaN
2,57627,Institut national d'études démographiques,institution,INED,NaN,NaN,NaN,NaN
3,205177,Arts des Images et Art Contemporain,laboratory,AIAC,UP8,11141,Université Paris 8 Vincennes-Saint-Denis,institution
4,441569,Centre National de la Recherche Scientifique,regroupinstitution,CNRS,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
2518,1179758,Laboratoire d'Analyses Génomiques des Eucaryotes,researchteam,LAGE,UMR 8030,21582,Génomique métabolique,department
2519,361641,Musée du quai Branly – Jacques Chirac,institution,MQBJC,NaN,NaN,NaN,NaN
2520,323000,Universidad Nacional de Colombia [Bogotà],institution,UNAL,NaN,NaN,NaN,NaN
2521,1096330,Institut Agro Montpellier,institution,NaN,Institut Agro,1042499,Institut national d'enseignement supérieur pou...,regroupinstitution


In [5]:
dfa.loc[dfa.idHal_s.isna(), "author_name"] = dfa.firstName_s.str.lower() +"-"+dfa.lastName_s.str.lower()
dfa.loc[~dfa.idHal_s.isna(), "author_name"] = dfa.idHal_s
dfa

,idHal_i,idHal_s,lastName_s,firstName_s,affPref_i,accountAssociated_bool,form_i,hasCV_bool,author_name
0,176081,jerome-livet,Livet,Jérôme,301475|205557,True,28862,True,jerome-livet
1,15426,francois-capron,Capron,François,NaN,True,7952,False,francois-capron
2,18849,philippe-blanchard,Blanchard,Philippe,205557|301475,True,31854,True,philippe-blanchard
3,245,jpchimier,Chimier,Jean-Philippe,301475|199947,True,24307,True,jpchimier
4,6836,agnes-couderc,Couderc,Agnès,301475|199947,True,24237,True,agnes-couderc
...,...,...,...,...,...,...,...,...,...
5887,5014,giurfa-martin,Martin,Grégory,NaN,True,420399,False,giurfa-martin
5888,754531,chantal-claudel,Claudel,Chantal,1057|107737,True,38508,True,chantal-claudel
5889,1316897,brigitte-bardin,Bardin,Brigitte,103953|11141|1180099,True,955148,False,brigitte-bardin
5890,753990,annette-burguet,Burguet,Anne,91848,True,56017,False,annette-burguet


In [6]:
df.loc[df.authors.str.lower().str.contains("luneau")]

,id,hal_id,pub_year,authors,authhalid_s,authhalid_i,orcid,struchal_id,hal_classif,bso_classif,pub_type,is_oa
1556,halhal-02407050,hal-02407050,2018,Johanna Lhuillier|Julio Bendezu-Sarmiento|Sama...,johanna-lhuillier|julio-cesar-bendezu-sarmient...,2468|180705|no_id_hal_i|no_id_hal_i|no_id_hal_...,0000-0003-2677-236X|0000-0002-0831-3955|no orc...,7550|199318270|118105|143987|180044174|11141|1...,shs.archeo_,Humanities,book-chapter,False
3811,halhal-03670303,hal-03670303,2022,Alexis Aulagnier|Lidia Chavinskaia|Lucile Garç...,alexis-aulagnier|no_id_hal_s|no_id_hal_s|aymer...,748590|no_id_hal_i|no_id_hal_i|965723|737628|n...,no orcid|no orcid|no orcid|0000-0002-6392-4994...,197534316|213191|580722|199318270|577435|42812...,shs_|shs.anthro-se_,Social sciences,book-chapter,True
30861,halhalshs-01492677,halshs-01492677,2016,Aymeric Luneau|Francis Chateauraynaud,aymeric-luneau|no_id_hal_s,965723|no_id_hal_i,0000-0002-6392-4994|no orcid,213191|199318270|11141|99539|441569|92500|15786,shs.socio_|sdv.spee_,Medical research,journal-article,True
36424,halhalshs-01494620,halshs-01494620,2016,Régine Boutrais|Aymeric Luneau,no_id_hal_s|aymeric-luneau,no_id_hal_i|965723,no orcid|0000-0002-6392-4994,213191|199318270|11141|301715|441569|15786|130...,shs.socio_,Social sciences,other,False
37519,haltel-00776198,tel-00776198,2010,Elise Luneau,elise-luneau,752175,0000-0003-3738-0142,7550|199318270|11141|300125|1003|441569|116205,shs.archeo_,Humanities,thesis,True
38083,halhalshs-01111864,halshs-01111864,2013,Julio Bendezu-Sarmiento|Johanna Lhuillier|Elis...,julio-cesar-bendezu-sarmiento|johanna-lhuillie...,180705|2468|752175,0000-0002-0831-3955|0000-0003-2677-236X|0000-0...,7550|199318270|130026149|11141|110691|226098|3...,shs.archeo_,Humanities,proceedings,False


# Jonction Auteur-Structure

In [7]:
dfs.columns

Index(['docid', 'name_s', 'type_s', 'acronym_s', 'parentAcronym_s',
       'parentDocid_i', 'parentName_s', 'parentType_s'],
      dtype='object')

In [8]:
dfs

,docid,name_s,type_s,acronym_s,parentAcronym_s,parentDocid_i,parentName_s,parentType_s
0,1002699,Centre d’histoire des sociétés Médiévales et M...,laboratory,MéMo,UP8|UPN,11141|116205,Université Paris 8 Vincennes-Saint-Denis|Unive...,institution|institution
1,11141,Université Paris 8 Vincennes-Saint-Denis,institution,UP8,NaN,NaN,NaN,NaN
2,57627,Institut national d'études démographiques,institution,INED,NaN,NaN,NaN,NaN
3,205177,Arts des Images et Art Contemporain,laboratory,AIAC,UP8,11141,Université Paris 8 Vincennes-Saint-Denis,institution
4,441569,Centre National de la Recherche Scientifique,regroupinstitution,CNRS,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
2518,1179758,Laboratoire d'Analyses Génomiques des Eucaryotes,researchteam,LAGE,UMR 8030,21582,Génomique métabolique,department
2519,361641,Musée du quai Branly – Jacques Chirac,institution,MQBJC,NaN,NaN,NaN,NaN
2520,323000,Universidad Nacional de Colombia [Bogotà],institution,UNAL,NaN,NaN,NaN,NaN
2521,1096330,Institut Agro Montpellier,institution,NaN,Institut Agro,1042499,Institut national d'enseignement supérieur pou...,regroupinstitution


In [9]:
dfa['docid'] = dfa.affPref_i.str.split("|") #docid correspond au nom de la colonne contenant les id des structures dans dfs
dfa1 = dfa.explode("docid")
dfa2 = dfa1[['idHal_i', 'idHal_s','docid']].dropna()

dfas0 = dfa2.merge(dfs, on = ["docid"], how = "left")
dfas = dfas0.loc[~dfas0.name_s.isna()].rename(columns={"idHal_i":"id"})
dfas["struct_id"] = dfas["docid"] +"_str"
dfas["author_id"] = dfas["id"] +"_au"


In [10]:
dfas.loc[dfas.idHal_s.str.lower().str.contains("luneau")]

,id,idHal_s,docid,name_s,type_s,acronym_s,parentAcronym_s,parentDocid_i,parentName_s,parentType_s,struct_id,author_id
2246,965723,aymeric-luneau,394361,médialab (Sciences Po),laboratory,médialab,Sciences Po,301587,Sciences Po,institution,394361_str,965723_au


On "explose" le dataframe concernant les publi pour avoir une ligne par co-auteur. permets de faire ensuite la liste des edges

In [11]:
df["author_id"] = df.authhalid_i.str.split("|")
df1 = df.explode("author_id")



liste_halid_i = [x for x in df1[["author_id"]].author_id.drop_duplicates() if x is not np.nan]
df2 = df1[["hal_id", "author_id"]].loc[df1.authhalid_i!="no_id_hal_i_au"].sort_values("hal_id")
df2["author_id"] = df2["author_id"] +"_au"
df2.loc[df2.author_id=="965723_au"]


,hal_id,author_id
3811,hal-03670303,965723_au
30861,halshs-01492677,965723_au
36424,halshs-01494620,965723_au


In [12]:
dfalab = dfas.loc[dfas.type_s=="laboratory"].rename(columns={"docid":"labhal_id",'acronym_s':'lab_acronym',"name_s":"lab_name"})#.dropna()
dfauniv = dfas.loc[dfas.type_s=="institution"].rename(columns={"docid":"instithal_id",'acronym_s':'instit_acronym', "name_s":"instit_name"})#.dropna()
dfas.loc[dfas.author_id=="965723_au"]
#df2b.loc[~(df2b.id.isna())|~(df2b.idHal_s.isna())]

,id,idHal_s,docid,name_s,type_s,acronym_s,parentAcronym_s,parentDocid_i,parentName_s,parentType_s,struct_id,author_id
2246,965723,aymeric-luneau,394361,médialab (Sciences Po),laboratory,médialab,Sciences Po,301587,Sciences Po,institution,394361_str,965723_au


In [13]:
df2b = df2.merge(dfalab, on =["author_id"], how ="left").dropna()
df2b

,hal_id,author_id,id,idHal_s,labhal_id,lab_name,type_s,lab_acronym,parentAcronym_s,parentDocid_i,parentName_s,parentType_s,struct_id
0,cea-00726543,853634_au,853634,pascal-omnes,78,"Laboratoire Analyse, Géométrie et Applications",laboratory,LAGA,UP8|UP13|CNRS,11141|15786|301340|441569,Université Paris 8 Vincennes-Saint-Denis|Unive...,institution|institution|institution|regroupins...,78_str
2,cea-01847290,1288809_au,1288809,khaldoun-zreik,1003467,Laboratoire Paragraphe,laboratory,PARAGRAPHE,UP8|CY,11141|1003413,Université Paris 8 Vincennes-Saint-Denis|CY Ce...,institution|institution,1003467_str
8,cea-02328190,7127_au,7127,gaspardpages,57701,Institut Français du Proche-Orient,laboratory,IFPO,MEAE|CNRS,301498|441569,Ministère de l'Europe et des Affaires étrangèr...,institution|regroupinstitution,57701_str
12,cea-02331768,180840_au,180840,stephanie-leroy,1095546,Institut de Recherche sur les Archéomatériaux,laboratory,IRAMAT,UTBM|UO|CNRS,300261|300297|419361|441569,Université de Technologie de Belfort-Montbelia...,institution|institution|institution|regroupins...,1095546_str
16,cea-02331768,12441_au,12441,maxime-lheritier,1003,Archéologies et Sciences de l'Antiquité,laboratory,ArScAn,UP1|UP8|UPN|MCC|CNRS,7550|11141|116205|300125|441569,Université Paris 1 Panthéon-Sorbonne|Universit...,institution|institution|institution|regroupins...,1003_str
...,...,...,...,...,...,...,...,...,...,...,...,...,...
74806,tel-04705261,1263179_au,1263179,sabrina-mazigh,226789,Centre de Recherche sur les Médiations,laboratory,Crem,UL,413289,Université de Lorraine,institution,226789_str
74807,tel-04706815,1303655_au,1303655,sebastien-broca,147868,"Centre d'études sur les médias, les technologi...",laboratory,CEMTI,UP8,11141,Université Paris 8 Vincennes-Saint-Denis,institution,147868_str
74810,tel-04746211,1426668_au,1426668,marie-claude-jolly-saad,1003,Archéologies et Sciences de l'Antiquité,laboratory,ArScAn,UP1|UP8|UPN|MCC|CNRS,7550|11141|116205|300125|441569,Université Paris 1 Panthéon-Sorbonne|Universit...,institution|institution|institution|regroupins...,1003_str
74812,tel-04762942,1236218_au,1236218,yves-citton,1007102,Fabrique du littéraire,laboratory,Fablitt,UP8,11141,Université Paris 8 Vincennes-Saint-Denis,institution,1007102_str


In [19]:
from scipy.stats import kstest

from numpy.random import seed
from numpy.random import poisson
from numpy.random import randn
from numpy.random import lognormal

from scipy.stats import chisquare


In [26]:
nb_isoa = df2.groupby(["is_oa"])["is_oa"].count().reset_index(name='count')
df_count = df2.groupby(["author_id","is_oa"]).agg(nb=("hal_id", "size" )).reset_index()
df_count

,author_id,is_oa,nb
0,1000068_au,False,24
1,1000068_au,True,4
2,10004_au,False,61
3,10004_au,True,3
4,10016_au,False,2
...,...,...,...
7714,9979_au,True,1
7715,9992_au,False,1
7716,999405_au,False,1
7717,no_id_hal_i_au,False,81218


In [31]:
freq_1 = df_count['nb'].loc[df_count.author_id=="1000068_au"].values
freq_g1 = nb_isoa['count'].values
sum(freq_1)
freq_g=freq_g1/sum(freq_g1)*sum(freq_1)
freq_g

array([24.51426169,  3.48573831])

In [33]:
statistics_topic={}
for aut in df_count["author_id"].unique():
    freq1  = df_count.loc[df_count["author_id"]==aut]
    freq_1=freq1['nb'].values
    if len(freq_1)==len(freq_g):
        ch=chisquare(f_obs=freq_1/sum(freq_1)*sum(freq_g), f_exp=freq_g)
        if ch.pvalue < .5:
            statistics_topic[aut]=ch.statistic
#            print(topic,ch)

In [35]:
a1_sorted_keys = sorted(statistics_topic, key=statistics_topic.get, reverse=True)
for r in a1_sorted_keys:
    print(r, statistics_topic[r])

171170_au 183.110079316079
171234_au 183.110079316079
12996_au 171.34463994569026
5085_au 169.8054296165697
14208_au 168.06938659866378
741389_au 163.8340525821
13175_au 161.21441517495086
183355_au 161.21441517495086
3796_au 161.21441517495086
752682_au 161.21441517495086
19773_au 158.14569958124898
746625_au 154.502164787895
1326256_au 153.13124224509036
1700_au 150.1066206613338
10853_au 144.7013911510431
13929_au 144.7013911510431
16499_au 139.3952734717642
1201577_au 137.89744406297234
1343565_au 137.89744406297234
742109_au 137.89744406297234
747702_au 137.89744406297234
1024849_au 135.2217398466695
1106946_au 133.79183393586158
753365_au 133.79183393586158
1259254_au 129.08037360624175
1296665_au 129.08037360624175
13627_au 129.08037360624175
172170_au 129.08037360624175
172751_au 129.08037360624175
19566_au 129.08037360624175
749911_au 129.08037360624175
5197_au 126.20607621438685
175121_au 120.55457057126016
1257646_au 117.22580423535726
1283411_au 117.22580423535726
14805_au 

In [37]:
df1_n=nb_isoa.copy()
df1_n['count'] = df1_n['count'] / sum(df1_n['count'])
df1_n

,is_oa,count
0,False,0.875509
1,True,0.124491


In [45]:
freq_all=pd.DataFrame()
for r in a1_sorted_keys:
    freq1_n=df_count.loc[df_count.author_id==r].copy()
    freq1_n['count'] = freq1_n['nb'] / sum(freq1_n['nb'])
    freq_duo=df1_n.merge(freq1_n,on='is_oa').rename(columns={'count_x':'global','count_y':'local'}).set_index('is_oa')
    freq_duo['zscore']=np.log(freq_duo['local']/freq_duo['global'])
    #del (freq_duo['local'])
    #del (freq_duo['global'])
    #del (freq_duo['nb'])
    freq_all=pd.concat([freq_all,freq_duo],axis=0)
    freq_duo=df1_n.merge(freq1_n,on='is_oa').rename(columns={'count_x':'global','count_y':r}).set_index('is_oa')
    #freq_duo.plot(kind='bar')
    #plt.title("statistics= "+str(int(statistics_topic[r])))
    #plt.tight_layout()
    #plt.savefig('../image/freq_dist/'+r+'_'+category+'.png')

In [48]:
freq_all2 = freq_all.reset_index()
freq_isoa = freq_all2.loc[freq_all2.is_oa==True]
dict_zscore = dict(zip(freq_isoa.author_id, freq_isoa.zscore))
dict_nb = dict(zip(freq_isoa.author_id, freq_isoa.nb))

In [49]:
dict_openaccess = dict(zip(df.hal_id, df.is_oa))
df2["is_oa"] = df2.hal_id.map(dict_openaccess.get)


In [51]:
df_halid = df2b[["hal_id"]].rename(columns={"hal_id":"id"}).drop_duplicates()
df_halid["Type"] = "publi"
df_auth= df2b[["author_id","idHal_s","lab_acronym","parentAcronym_s", "parentName_s"]].drop_duplicates("author_id").rename(columns={"author_id":"id"})
df_auth["Type"] = "auteur"
df_auth["nb_oa"] = df_auth.id.map(dict_nb.get)
df_auth["zscore"] = df_auth.id.map(dict_zscore.get)
df_auth = df_auth.fillna(0)




## Rattachement institutionnel des labos à P8

for

In [53]:

df_auth.loc[df_auth.parentAcronym_s.str.lower().str.contains("up8") == True, "is_P8"] = True
df_auth.loc[df_auth.parentAcronym_s.str.lower().str.contains("up8") == False, "is_P8"] = False

nodes = pd.concat([df_halid, df_auth])
np.max(nodes.zscore)

2.051775936629855

# Graphe Auteur-Publi

In [54]:
G = nx.Graph()
G.add_nodes_from(df2b.hal_id, part="publi")
G.add_nodes_from(df2b.author_id, part="auteur")
G.add_edges_from(df2b[["hal_id", "author_id"]].values)



In [55]:
nodes["Label"] = nodes["id"]
node_attr = nodes.set_index('id').to_dict('index')
nx.set_node_attributes(G, node_attr)



In [ ]:
Sigma(G, node_color="is_P8")

In [56]:
monoG = pelote.monopartite_projection(G, 'auteur')
nx.number_of_nodes(monoG)
nx.number_connected_components(monoG)

df_auth["Label"] = df_auth["id"]
node_attr = df_auth.set_index('id').to_dict('index')
nx.set_node_attributes(monoG, node_attr)


In [57]:
degres = nx.degree_centrality(monoG)

not_normalized_degres = {}

for x in degres :
    not_normalized_degres[x] = degres[x] * (nx.number_of_nodes(monoG)-1)


nx.set_node_attributes(monoG, degres, 'normalized_degres')
nx.set_node_attributes(monoG, not_normalized_degres, 'degres')
#h = pelote.filter_nodes(monoG, lambda n, a: a["degres"] > 2)
#pelote.crop_to_largest_connected_component(monoG)
print(nx.number_connected_components(monoG))
louv_communities = nx.community.louvain_communities(monoG)

community={}

print(len(louv_communities))
for n, x in enumerate(louv_communities):
    #print(n)
    for name in x:
        community[name] = n

nx.set_node_attributes(monoG, community, 'community')   


1165
1190


In [58]:
Sigma(monoG, node_label='idHal_s', node_color= "is_P8", node_shape="type_node", node_size="nb_oa")


Sigma(nx.Graph with 3,578 nodes and 5,960 edges)

# Graphe Publi-labo

In [ ]:
df2bb = df2b[["hal_id","labhal_id", "lab_acronym","parentAcronym_s", "parentName_s"]].drop_duplicates()
G = nx.Graph()
G.add_nodes_from(df2bb.halId_s, part="publi")
G.add_nodes_from(df2bb.labhal_id, part="labos")
G.add_edges_from(df2b[["halId_s", "labhal_id"]].values)

In [ ]:
dict_openaccess = dict(zip(df.halId_s, df.openAccess_bool))
df2bb["is_oa"] = df2bb.halId_s.map(dict_openaccess.get)
freq_oa = df2bb.groupby(["labhal_id","is_oa"]).agg(nb=("labhal_id", "size" )).reset_index()
freq_total = df2bb.groupby(["labhal_id"]).agg(total=("labhal_id", "size" )).reset_index()
freq_oa =freq_oa.merge(freq_total, on = ["labhal_id"], how = "left")
freq_oa["ratio_oa"] = (freq_oa.nb / freq_oa.total)
freq_oa["log_ratio_oa"] = np.log(1 + freq_oa.nb)
freq_oa["log_ratio_oa1"] = freq_oa["log_ratio_oa"].astype(int)
freq_oa["log_ratio_oa2"] = freq_oa["log_ratio_oa"].round(1)#astype(int)

dict_freq_oa = dict(zip(freq_oa.labhal_id.loc[freq_oa.is_oa==True], freq_oa.nb.loc[freq_oa.is_oa==True]))
dict_total = dict(zip(freq_oa.labhal_id.loc[freq_oa.is_oa==True], freq_oa.total.loc[freq_oa.is_oa==True]))
dict_ratio_oa = dict(zip(freq_oa.labhal_id.loc[freq_oa.is_oa==True], freq_oa.ratio_oa.loc[freq_oa.is_oa==True]))
dict_log_oa = dict(zip(freq_oa.labhal_id.loc[freq_oa.is_oa==True], freq_oa.log_ratio_oa.loc[freq_oa.is_oa==True]))
dict_log_oa2 = dict(zip(freq_oa.labhal_id.loc[freq_oa.is_oa==True], freq_oa.log_ratio_oa1.loc[freq_oa.is_oa==True]))
dict_log_oa3 = dict(zip(freq_oa.labhal_id.loc[freq_oa.is_oa==True], freq_oa.log_ratio_oa2.loc[freq_oa.is_oa==True]))
freq_oa

In [ ]:
df_halid = df2bb[["halId_s"]].rename(columns={"halId_s":"id"}).drop_duplicates()
df_halid["Type"] = "publi"
df_auth= df2bb[["labhal_id", "lab_acronym","parentAcronym_s", "parentName_s"]].drop_duplicates().rename(columns={"labhal_id":"id"})
df_auth["Type"] = "labos"

df_auth.loc[df_auth.parentAcronym_s.str.lower().str.contains("up8") == True, "is_P8"] = True
df_auth.loc[df_auth.parentAcronym_s.str.lower().str.contains("up8") == False, "is_P8"] = False


df_auth["nb_oa"] = df_auth.id.map(dict_freq_oa.get)
df_auth["ratio_oa"] = df_auth.id.map(dict_ratio_oa.get)
df_auth["log_ratio_oa"] = df_auth.id.map(dict_log_oa.get)
df_auth["log_ratio_oa1"] = df_auth.id.map(dict_log_oa2.get)
df_auth["log_ratio_oa2"] = df_auth.id.map(dict_log_oa3.get)
df_auth["total"] = df_auth.id.map(dict_total.get)

nodes = pd.concat([df_halid, df_auth])
df_auth

In [ ]:
nodes["Label"] = nodes["id"]
node_attr = nodes.set_index('id').to_dict('index')
nx.set_node_attributes(G, node_attr)



In [ ]:
monoG = pelote.monopartite_projection(G, 'labos')
nx.number_of_nodes(monoG)
nx.number_connected_components(monoG)

df_auth["Label"] = df_auth["id"]
node_attr = df_auth.set_index('id').to_dict('index')
nx.set_node_attributes(monoG, node_attr)


In [ ]:
degres = nx.degree_centrality(monoG)

not_normalized_degres = {}

for x in degres :
    not_normalized_degres[x] = degres[x] * (nx.number_of_nodes(monoG)-1)


nx.set_node_attributes(monoG, degres, 'normalized_degres')
nx.set_node_attributes(monoG, not_normalized_degres, 'degres')
#h = pelote.filter_nodes(monoG, lambda n, a: a["degres"] > 2)
pelote.crop_to_largest_connected_component(monoG)
print(nx.number_connected_components(monoG))
louv_communities = nx.community.louvain_communities(monoG)

community={}

print(len(louv_communities))
for n, x in enumerate(louv_communities):
    #print(n)
    for name in x:
        community[name] = n

nx.set_node_attributes(monoG, community, 'community')   


In [ ]:
Sigma(monoG, node_label='lab_acronym', node_color= "is_P8", node_shape="Type", node_size="nb_oa")


## Graphe auteur et institution

In [ ]:

dfasb = dfas.loc[(dfas.type_s=="laboratory")]
dfasb.loc[(dfasb.acronym_s.isna()), "acronym"] = dfasb.docid
dfasb.loc[~(dfasb.acronym_s.isna()), "acronym"] = dfasb.acronym_s
dfasb.loc[(dfasb.idHal_s.isna()), "idHal"] = dfasb.id
dfasb.loc[~(dfasb.idHal_s.isna()), "idHal"] = dfasb.idHal_s
dfas_str = dfasb[["acronym","type_s"]].rename(columns={"acronym":"nom"}).drop_duplicates()

dfas_str["Type"] = "structure"
dfas_au= dfasb[["idHal"]].rename(columns={"idHal":"nom"}).drop_duplicates()

dfas_au["Type"] = "chercheur"
dfas_au["type_s"] = "chercheur"
nodes = pd.concat([dfas_au, dfas_str]).dropna()
nodes
#nodes = nodes0.merge(dfalab[['id', 'idHal_s', 'labhal_id', 'lab_name','lab_acronym']], on =["id"], how ="left")#.merge(dfauniv[['id', 'instithal_id', 'instit_name',
       #'instit_acronym']], on =["id"], how ="left")
#nodes = nodes0.merge(dfauniv[['id', 'idHal_s','instithal_id', 'instit_name', 'instit_acronym']], on =["id"], how ="left")
l = [x for x in dfasb.idHal if isinstance(x, float)==True]
nodes

In [ ]:
G = nx.Graph()
G.add_nodes_from(dfasb.acronym, part="structure")
G.add_nodes_from(dfasb.idHal, part="chercheur")
G.add_edges_from(dfasb[["idHal", "acronym"]].values)

In [ ]:
nodes["Label"] = nodes["nom"]
node_attr = nodes.set_index('nom').to_dict('index')
nx.set_node_attributes(G, node_attr)



In [ ]:
monoG = pelote.monopartite_projection(G, 'chercheur')
nx.number_of_nodes(monoG)
nx.number_connected_components(monoG)

In [ ]:
degres = nx.degree_centrality(monoG)

not_normalized_degres = {}

for x in degres :
    not_normalized_degres[x] = degres[x] * (nx.number_of_nodes(monoG)-1)


nx.set_node_attributes(monoG, degres, 'normalized_degres')
nx.set_node_attributes(monoG, not_normalized_degres, 'degres')
#h = pelote.filter_nodes(monoG, lambda n, a: a["degres"] > 2)
#pelote.crop_to_largest_connected_component(monoG)
print(nx.number_connected_components(monoG))
louv_communities = nx.community.louvain_communities(monoG)

community={}

print(len(louv_communities))
for n, x in enumerate(louv_communities):
    #print(n)
    for name in x:
        community[name] = n

nx.set_node_attributes(monoG, community, 'community')   


In [ ]:
Sigma(monoG, node_label="Label", node_color= "community", node_shape="type_s", node_size="degres")
